In [9]:
import twitter
import json
import time
import pandas as pd
import networkx as nx
import pickle
import random
import config2 as config
from pytwitter import Api
import os.path

In [2]:
twitter_api = twitter.Api(consumer_key=config.twitter_consumer_key,
                              consumer_secret=config.twitter_consumer_secret, 
                              access_token_key=config.twitter_access_token, 
                              access_token_secret=config.twitter_access_secret,
                              sleep_on_rate_limit=True)

In [3]:
twitter_api2 = twitter.Api(consumer_key=config.twitter_consumer_key,
                              consumer_secret=config.twitter_consumer_secret, 
                              access_token_key=config.twitter_access_token, 
                              access_token_secret=config.twitter_access_secret)

In [4]:
most_followed = pd.read_csv('top100.csv')
most_followed

,rank,name,followers_count,account
0,1st,BTS,47807327,BTS_twt
1,2nd,EXO,13348657,weareoneEXO
2,3rd,TWICE,11012422,JYPETWICE
3,4th,SEVENTEEN,10747186,pledis_17
4,5th,TXT,10280735,TXT_bighit
...,...,...,...,...
95,101st,Cherry Bullet,211229,cherrybullet
96,102nd,N.Flying,205723,NFlyingofficial
97,103rd,ONF,203267,WM_ONOFF
98,104th,Girl's Day,202816,Girls_Day


In [5]:
def get_screenname(most_followed):
    for i in most_followed.index:
        accounts = twitter_api.GetUsersSearch(most_followed['name'][i])
        for a in accounts:
            if abs(a.followers_count - most_followed['followers_count'][i]) < 10000:
                most_followed['account'][i] = a.screen_name
                break
        print('{}/{} {}'.format(i+1, len(most_followed), most_followed['account'][i]))

In [6]:
most_followed

,rank,name,followers_count,account
0,1st,BTS,47807327,BTS_twt
1,2nd,EXO,13348657,weareoneEXO
2,3rd,TWICE,11012422,JYPETWICE
3,4th,SEVENTEEN,10747186,pledis_17
4,5th,TXT,10280735,TXT_bighit
...,...,...,...,...
95,101st,Cherry Bullet,211229,cherrybullet
96,102nd,N.Flying,205723,NFlyingofficial
97,103rd,ONF,203267,WM_ONOFF
98,104th,Girl's Day,202816,Girls_Day


In [7]:
dir(twitter_api)

['CheckRateLimit',
 'ClearCredentials',
 'CreateBlock',
 'CreateFavorite',
 'CreateFriendship',
 'CreateList',
 'CreateListsMember',
 'CreateMute',
 'CreateSubscription',
 'DEFAULT_CACHE_TIMEOUT',
 'DestroyBlock',
 'DestroyDirectMessage',
 'DestroyFavorite',
 'DestroyFriendship',
 'DestroyList',
 'DestroyListsMember',
 'DestroyMute',
 'DestroyStatus',
 'DestroySubscription',
 'GetAppOnlyAuthToken',
 'GetBlocks',
 'GetBlocksIDs',
 'GetBlocksIDsPaged',
 'GetBlocksPaged',
 'GetDirectMessages',
 'GetFavorites',
 'GetFollowerIDs',
 'GetFollowerIDsPaged',
 'GetFollowers',
 'GetFollowersPaged',
 'GetFriendIDs',
 'GetFriendIDsPaged',
 'GetFriends',
 'GetFriendsPaged',
 'GetHelpConfiguration',
 'GetHomeTimeline',
 'GetListMembers',
 'GetListMembersPaged',
 'GetListTimeline',
 'GetLists',
 'GetListsList',
 'GetListsPaged',
 'GetMemberships',
 'GetMentions',
 'GetMutes',
 'GetMutesIDs',
 'GetMutesIDsPaged',
 'GetMutesPaged',
 'GetReplies',
 'GetRetweeters',
 'GetRetweets',
 'GetRetweetsOfMe',
 'G

In [9]:
followers = twitter_api.GetFollowerIDs(screen_name='by_verivery')
followers

[1580214621318832130,
 1369584763502096385,
 1590846287296937984,
 1335244646918672386,
 1579839146147090432,
 1490202440607711233,
 1572327133996777474,
 1589766172475809794,
 2267297461,
 1590854912027435008,
 1590592991101870081,
 1524015370717532160,
 1067139041580204038,
 1590828421390929921,
 1001864863092826112,
 995576516208553984,
 1524002850560688135,
 1292035723193978880,
 1412502992448860163,
 1275205241542266881,
 1590820462719901713,
 1438384445040386056,
 101481892,
 3842652682,
 1308941417620942849,
 1580980579889287169,
 1565220298684760066,
 1535732097943408640,
 1540723556920659971,
 1461819214797418499,
 1590795521471483933,
 1264638663381696514,
 1390688234519289862,
 1483891079686537221,
 949829983417110529,
 942838617201872896,
 1590774176344068096,
 855485180,
 1554501309373353984,
 1577680028103761920,
 1590743969943752706,
 1589773448305418247,
 978317435664195584,
 1590759387001491461,
 1588340287859998720,
 1564605677389348877,
 1589535351403520000,
 1588152

In [ ]:
follower0 = twitter_api.GetFollowers(screen_name='by_verivery', count=10)

In [ ]:
def collecting_followers(account):
    f_ids = twitter_api.GetFollowerIDs(screen_name=account, total_count=5000)
    return f_ids

In [ ]:
f_dict = {}
    for i in most_followed.index:
        f_dict[most_followed['account'][i]] = collecting_followers(most_followed['account'][i])
        print('{}/{} complete'.format(i+1, len(most_followed)))

In [14]:
def collecting_mention(f_dict):
    i = len(f_dict)
    i0 = 0
    for key in f_dict:
        if not os.path.isfile('C:/Users/rubby/Documents/GitHub/twitter-fandom-analysis/data/usrNet/{}.net'.format(key)):
            G = nx.Graph()
            i0 += 1
            followers = f_dict[key]
            j = len(followers)
            j0 = 0
            for follower in followers:
                j0 += 1
                print('key: {}/{}, follower: {}/{}'.format(i0,i,j0,j))
                try:
                    statuses = twitter_api.GetUserTimeline(follower)
                except Exception as e:
                    print(e)
                    continue
                for status in statuses:
                    if status.user_mentions:
                        for m in status.user_mentions:
                            try:
                                following = twitter_api.ShowFriendship(source_user_id=m.id, target_screen_name=key)
                            except Exception as e:
                                print(e)
                                continue
                            if following.get('relationship').get('source').get('following'):
                                G.add_edge(follower, m.id)
                                nx.write_pajek(G, 'C:/Users/rubby/Documents/GitHub/twitter-fandom-analysis/data/usrNet/{}.net'.format(key))
                                print('add edge of ' + key)
                    else:
                        continue
        else:
            i0 += 1
            print('{} network exists'.format(key))

In [7]:
with open("C:/Users/rubby/Documents/GitHub/twitter-fandom-analysis/followers.p", 'rb') as f:
    f_dict = pickle.load(f)

f_dict_sample = {}

random.seed(100)
for key, value in f_dict.items():
    followers_s = random.sample(value, 500)
    f_dict_sample[key] = followers_s

In [ ]:
collecting_mention(f_dict_sample)

@BTS_twt network exists
@weareoneEXO network exists
@JYPETWICE network exists
@GOT7Official network exists
@TXT_bighit network exists
@pledis_17 network exists
@NCTsmtown network exists
@NCTsmtown_127 network exists
@Stray_Kids network exists
@NCTsmtown_DREAM network exists
@BLACKPINK network exists
@Rvsmtown network exists
@OfficialMonstaX network exists
@ENHYPEN network exists
@ITZYofficial network exists
@offclASTRO network exists
@GirlsGeneration network exists
@ATEEZofficial network exists
@treasuremembers network exists
@superm network exists
@WannaOne_twt network exists
@RBW_MAMAMOO network exists
@aespa_official network exists
@day6official network exists
@SHINee network exists
@WayV_official network exists
@YG_GlobalVIP network exists
@IST_THEBOYZ network exists
@OFFICIALBTOB network exists
@SJofficial network exists
@G_I_DLE network exists
@official_CNBLUE network exists
@GFRDofficial network exists
@follow_2PM network exists
@official_izone network exists
@CUBE_PTG network e

In [9]:
def user_info(fgraphs):
    fan_ids = []
    for fg in fgraphs:
        fan_ids += list(fg.nodes)
    fan_ids = list(set(fan_ids))
    
    fan_info = {}
    i = 1
    fail_count = 0
    for fid in fan_ids:
        try:
            fan_info[fid] = twitter_api.GetUser(fid, return_json=True)
        except:
            fail_count += 1
            if fail_count % 10 == 0:
                print('{} users not found'.format(fail_count))
        with open("C:/Users/rubby/Documents/GitHub/twitter-fandom-analysis/users.json", 'w') as f:
            json.dump(fan_info, f, indent=4)
        if i % 100 == 0:
            print('{}/{} user info saved'.format(i, len(fan_ids)))
        i += 1